<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part I - 4a <br><br><br>
  Sentence Denoising
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

3. Language Modeling

4. <font color=red>**Sequence Labelling**</font>


### Part II

5. Auto-Encoding

6. Machine Translation

7. Text Classification




### Part III

8. Abstractive Summarization

9. Question Answering

10. Chatbot


</div>

***

<a id="plan"></a>

| | | | |
|------|------|------|------|
| **Content** | [Corpus](#corpus) | [Modules](#modules) | [Model](#model) | 


# Overview

We consider as Sequence labelling task a **Sentence Denoising** problem, which consists in transforming a noisy sequence of words into a correctly formed sentence.<br> Training follows a denoising objective known as _Cloze task_ , which is used :

- For the BERT model in [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)

# Packages

In [2]:
from __future__ import unicode_literals, print_function, division
import sys
import warnings
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 1.4.0
DL device : cuda


In [3]:
path_to_NLP = 'C:\\Users\\Jb\\Desktop\\NLP'

In [4]:
sys.path.append(path_to_NLP + '\\DL4NLP lib')

<a id="corpus"></a>

# Corpus

[Back to top](#plan)

Le texte est importé et mis sous forme de liste, où chaque élément représente un texte présenté sous forme d'une liste de mots.<br> Le corpus est donc une fois importé sous le forme :<br>

- corpus = [text]<br>
- text   = [word]<br>
- word   = str<br>

In [5]:
def cleanSentence(sentence): # -------------------------  str
    sw = ['']
    #sw += nltk.corpus.stopwords.words('english')
    #sw += nltk.corpus.stopwords.words('french')

    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join( c for c in unicodedata.normalize('NFD', s)
                        if unicodedata.category(c) != 'Mn')

    def normalizeString(s):
        '''Remove rare symbols from a string'''
        s = unicodeToAscii(s.lower().strip()) # 
        #s = re.sub(r"[^a-zA-Z\.\(\)\[\]]+", r" ", s)  # 'r' before a string is for 'raw' # ?&\%\_\- removed # set('''.,:;()*#&-_%!?/\'")''')
        return s

    def wordTokenizerFunction():
        # base version
        function = lambda sentence : sentence.strip().split()

        # nltk version
        #function = word_tokenize    
        return function

    # 1 - caractères spéciaux
    def clean_sentence_punct(text): # --------------  str
        text = normalizeString(text)
        # suppression de la dernière ponctuation
        if (len(text) > 0 and text[-1] in ['.', ',', ';', ':', '!', '?']) : text = text[:-1]

        text = text.replace(r'(', r' ( ')
        text = text.replace(r')', r' ) ')
        text = text.replace(r'[', r' [ ')
        text = text.replace(r']', r' ] ')
        text = text.replace(r'<', r' < ')
        text = text.replace(r'>', r' > ')

        text = text.replace(r':', r' : ')
        text = text.replace(r';', r' ; ')
        for i in range(5) :
            text = re.sub('(?P<val1>[0-9])\.(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
            text = re.sub('(?P<val1>[0-9]),(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
        text = text.replace(r',', ' , ')
        text = text.replace(r'.', ' . ')
        for i in range(5) : text = re.sub('(?P<val1>[p0-9])__-__(?P<val2>[p0-9])', '\g<val1>.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. p \. (?P<val2>[0-9])', '\g<val1>.p.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. s \. (?P<val2>[0-9])', '\g<val1>.s.\g<val2>', text)

        text = text.replace(r'"', r' " ')
        text = text.replace(r'’', r" ' ")
        text = text.replace(r'”', r' " ')
        text = text.replace(r'“', r' " ')
        text = text.replace(r'/', r' / ')

        text = re.sub('(…)+', ' … ', text)
        text = text.replace('≤', ' ≤ ')          
        text = text.replace('≥', ' ≥ ')
        text = text.replace('°c', ' °c ')
        text = text.replace('°C', ' °c ')
        text = text.replace('ºc', ' °c ')
        text = text.replace('n°', 'n° ')
        text = text.replace('%', ' % ')
        text = text.replace('*', ' * ')
        text = text.replace('+', ' + ')
        text = text.replace('-', ' - ')
        text = text.replace('_', ' ')
        text = text.replace('®', ' ')
        text = text.replace('™', ' ')
        text = text.replace('±', ' ± ')
        text = text.replace('÷', ' ÷ ')
        text = text.replace('–', ' - ')
        text = text.replace('μg', ' µg')
        text = text.replace('µg', ' µg')
        text = text.replace('µl', ' µl')
        text = text.replace('μl', ' µl')
        text = text.replace('µm', ' µm')
        text = text.replace('μm', ' µm')
        text = text.replace('ppm', ' ppm')
        text = re.sub('(?P<val1>[0-9])mm', '\g<val1> mm', text)
        text = re.sub('(?P<val1>[0-9])g', '\g<val1> g', text)
        text = text.replace('nm', ' nm')

        text = re.sub('fa(?P<val1>[0-9])', 'fa \g<val1>', text)
        text = re.sub('g(?P<val1>[0-9])', 'g \g<val1>', text)
        text = re.sub('n(?P<val1>[0-9])', 'n \g<val1>', text)
        text = re.sub('p(?P<val1>[0-9])', 'p \g<val1>', text)
        text = re.sub('q_(?P<val1>[0-9])', 'q_ \g<val1>', text)
        text = re.sub('u(?P<val1>[0-9])', 'u \g<val1>', text)
        text = re.sub('ud(?P<val1>[0-9])', 'ud \g<val1>', text)
        text = re.sub('ui(?P<val1>[0-9])', 'ui \g<val1>', text)

        text = text.replace('=', ' ')
        text = text.replace('!', ' ')
        text = text.replace('-', ' ')
        text = text.replace(r' , ', ' ')
        text = text.replace(r' . ', ' ')

        text = re.sub('(?P<val>[0-9])ml', '\g<val> ml', text)
        text = re.sub('(?P<val>[0-9])mg', '\g<val> mg', text)

        for i in range(5) : text = re.sub('( [0-9]+ )', ' ', text)
        #text = re.sub('cochran(\S)*', 'cochran ', text)
        return text

    # 3 - split des mots
    def wordSplit(sentence, tokenizeur): # ------------- [str]
        return tokenizeur(sentence)

    # 4 - mise en minuscule et enlèvement des stopwords
    def stopwordsRemoval(sentence, sw): # ------------- [[str]]
        return [word for word in sentence if word not in sw]

    # 6 - correction des mots
    def correction(text):
        def correct(word):
            return spelling.suggest(word)[0]
        list_of_list_of_words = [[correct(word) for word in sentence] for sentence in text]
        return list_of_list_of_words

    # 7 - stemming
    def stemming(text): # ------------------------- [[str]]
        list_of_list_of_words = [[PorterStemmer().stem(word) for word in sentence if word not in sw] for sentence in text]
        return list_of_list_of_words


    tokenizeur = wordTokenizerFunction()
    sentence = clean_sentence_punct(str(sentence))
    sentence = wordSplit(sentence, tokenizeur)
    sentence = stopwordsRemoval(sentence, sw)
    #text = correction(text)
    #text = stemming(text)
    return sentence


def importWords(file_name) :
    def cleanDatabase(db):
        words = ['.']
        title = ''
        for pair in db :
            #print(pair)
            current_tile = pair[0].split(' | ')[-1]
            if current_tile != title :
                words += cleanSentence(current_tile) + ['.']
                title  = current_tile
            words += cleanSentence(str(pair[1]).split(' | ')[-1]) + ['.']
        return words

    df = pd.read_excel(file_name, sep = ',', header = None)
    headers = [i for i, titre in enumerate(df.iloc[0,:].values) if i in [1, 2] or titre == 'score manuel'] 
    db = df.iloc[1:, headers].values.tolist()
    db = [el[:2] for el in db if el[-1] in [0,1, 10]]
    words = cleanDatabase(db)
    return words


def importAllWords(path_to_data) :
    corpus = []
    reps = os.listdir(path_to_data)
    for rep in reps :
        files = os.listdir(path_to_data + '\\' + rep)
        for file in files :
            file_name = path_to_data + '\\' + rep + '\\' + file
            corpus.append(importWords(file_name))
    return corpus



def importSentences(file_name) :
    def cleanDatabase(db):
        sentences = []
        title = ''
        for pair in db :
            current_tile = pair[0].split(' | ')[-1]
            if current_tile != title :
                sentences.append(cleanSentence(current_tile))
                title = current_tile
            sentences.append(cleanSentence(str(pair[1]).split(' | ')[-1]))
        return sentences

    df = pd.read_excel(file_name, sep = ',', header = None)
    headers = [i for i, titre in enumerate(df.iloc[0,:].values) if i in [1, 2] or titre == 'score manuel'] 
    db = df.iloc[1:, headers].values.tolist()
    db = [el[:2] for el in db if el[-1] in [0,1, 10]]
    sentences = cleanDatabase(db)
    return sentences


def importAllSentences(path_to_data) :
    corpus = []
    reps = os.listdir(path_to_data)
    for rep in reps :
        files = os.listdir(path_to_data + '\\' + rep)
        for file in files :
            file_name = path_to_data + '\\' + rep + '\\' + file
            corpus += importSentences(file_name)
    return corpus

In [6]:
corpus = importAllWords(path_to_NLP + '\\data\\AMM')
len(corpus)

510

In [7]:
sentences = importAllSentences(path_to_NLP + '\\data\\AMM')
len(sentences)

31574

In [8]:
sentences[10]

['this',
 'test',
 'consists',
 'of',
 'assessing',
 'the',
 'dissolution',
 'time',
 'of',
 'the',
 'freeze',
 'dried',
 'yellow',
 'fever',
 'vaccine',
 'after',
 'adding',
 'the',
 'suitable',
 'diluent',
 'directly',
 'into',
 'the',
 'original',
 'container']

<a id="modules"></a>

# 1 Modules

### 1.1 Word Embedding module

[Back to top](#plan)

All details on Word Embedding modules and their pre-training are found in **Part I - 1**. We consider here a FastText model trained following the Skip-Gram training objective.

In [9]:
from libDL4NLP.models.Word_Embedding import Word2Vec as myWord2Vec
from libDL4NLP.models.Word_Embedding import Word2VecConnector
from libDL4NLP.utils.Lang import Lang

In [10]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath, get_tmpfile

In [33]:
word2vec = FastText(size = 75, 
                    window = 5, 
                    min_count = 3, 
                    negative = 20,
                    sg = 1)

In [34]:
word2vec.build_vocab(sentences)

In [35]:
len(word2vec.wv.vocab)

4662

In [36]:
word2vec.train(sentences = sentences, 
               epochs = 50,
               total_examples = word2vec.corpus_count)

In [37]:
# save
#word2vec.save(get_tmpfile(path_to_NLP + '\\saves\\models\\DL4NLP_I4a_word2vec.model'))

# load trained model
#word2vec = FastText.load(get_tmpfile(path_to_NLP + '\\saves\\models\\DL4NLP_I4a_word2vec.model'))

### 1.2 Contextualization module

[Back to top](#plan)


In [20]:
from libDL4NLP.modules import RecurrentEncoder

<a id="model"></a>

# 2 Sentence denoising Model

[Back to top](#plan)


In [21]:
class SentenceDenoiser(nn.Module) :
    def __init__(self, device, tokenizer, word2vec, 
                 hidden_dim = 100, 
                 n_layers = 1, 
                 dropout = 0, 
                 class_weights = None, 
                 optimizer = optim.SGD
                 ):
        super(SentenceDenoiser, self).__init__()
        
        # embedding
        self.tokenizer = tokenizer
        self.word2vec  = word2vec
        self.context   = RecurrentEncoder(self.word2vec.output_dim, hidden_dim, n_layers, dropout, bidirectional = True)
        self.out       = nn.Linear(self.context.output_dim, self.word2vec.lang.n_words)
        self.act       = F.softmax
        
        # optimizer
        self.ignore_index = self.word2vec.lang.getIndex('PADDING_WORD')
        self.criterion = nn.NLLLoss(size_average = False, 
                                    ignore_index = self.ignore_index, 
                                    weight = class_weights)
        self.optimizer = optimizer
        
        # load to device
        self.device = device
        self.to(device)
        
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def predict_proba(self, words):
        embeddings = self.word2vec.twin(words, self.device) # dim = (1, input_length, hidden_dim)
        hiddens, _ = self.context(embeddings)               # dim = (1, input_length, hidden_dim)
        probs      = self.act(self.out(hiddens), dim = 2)   # dim = (1, input_length, lang_size)
        return probs

    # main method
    def forward(self, sentence = '.', color = '\033[94m'):
        def addColor(w1, w2, color) : return color + w2 + '\033[0m' if w1 != w2 else w2
        words  = self.tokenizer(sentence)
        probs  = self.predict_proba(words).squeeze(0) # dim = (input_length, lang_size)
        inds   = [probs[i].data.topk(1)[1].item() for i in range(probs.size(0))]
        new_ws = [self.word2vec.lang.index2word[ind] for ind in inds]
        print(' '.join([addColor(w1, w2, color) for w1, w2 in zip(words, new_ws)]))
        return

    # load data
    def generatePackedSentences(self, 
                                sentences, 
                                batch_size = 32, 
                                mask_ratio = 0.15,
                                max_sentence_length = 50,
                                tol = 10,
                                seed = 42) :
        def maskInput(index, b) :
            if   b and random.random() > 0.25 : return self.word2vec.lang.getIndex('UNK')
            elif b and random.random() > 0.10 : return random.choice(list(self.word2vec.twin.lang.word2index.values()))
            else                              : return index
            
        def maskOutput(index, b) :
            return index if b else self.ignore_index
        
        def splitLongs(words, threshold = 50, tol = 10):
            news = []
            for i in range(0, len(words), threshold) :
                if len(words)-i-threshold > tol : 
                    news.append(words[i : i + threshold])
                else : 
                    news.append(words[i:])
                    break
            return news
        
        packed_data = []
        random.seed(seed)
        # prepare sentences
        #sentences = [self.tokenizer(s) for s in sentences]
        sentences = [[self.word2vec.lang.getIndex(w) for w in s] for s in sentences]
        sentences = [[w for w in words if w is not None] for words in sentences]
        sentences = [s for S in sentences for s in splitLongs(S, max_sentence_length, tol) if len([w for w in s if w != self.word2vec.lang.getIndex('UNK')]) > 1]
        sentences.sort(key = lambda s: len(s), reverse = True)
        # collect packs
        for i in range(0, len(sentences), batch_size) :
            pack = sentences[i:i + batch_size]
            # prepare mask
            mask_xl = [[i for i, w in enumerate(p) if w != self.word2vec.lang.getIndex('UNK')] for p in pack]
            mask_xs = [random.sample(m, k = int(mask_ratio*len(m) +1)) for m in mask_xl]
            # prepare input and target packs
            pack0    = [[ maskInput(s[i], i in m) for i in range(len(s))] for s, m in zip(pack, mask_xs)]
            pack1_xs = [[maskOutput(s[i], i in m) for i in range(len(s))] for s, m in zip(pack, mask_xs)]
            pack1_xl = [[maskOutput(s[i], i in m) for i in range(len(s))] for s, m in zip(pack, mask_xl)]
            lengths  = torch.tensor([len(p) for p in pack0]) # size = (batch_size) 
            # padd
            pack0    = list(itertools.zip_longest(*pack0, fillvalue = self.ignore_index)) 
            pack1_xs = list(itertools.zip_longest(*pack1_xs, fillvalue = self.ignore_index))
            pack1_xl = list(itertools.zip_longest(*pack1_xl, fillvalue = self.ignore_index))
            # turn into torch variables
            pack0 = Variable(torch.LongTensor(pack0).transpose(0, 1))   # size = (batch_size, max_length)
            pack1_xs = Variable(torch.LongTensor(pack1_xs).transpose(0, 1))   # size = (batch_size, max_length) 
            pack1_xl = Variable(torch.LongTensor(pack1_xl).transpose(0, 1))   # size = (batch_size, max_length) 
            # store pack
            packed_data.append([[pack0, lengths], [pack1_xs, pack1_xl]])
        return packed_data
    
    # fit model
    def fit(self, batches, 
            iters = None, epochs = None, lr = 0.025, unmasked_ratio = 0,
            random_state = 42, print_every = 10, compute_accuracy = 'xs'):
        """Performs training over a given dataset and along a specified amount of loops"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
        
        def computeLogProbs(batch) :
            embeddings = self.word2vec.embedding(batch[0].to(self.device))
            hiddens,_  = self.context(embeddings, lengths = batch[1].to(self.device)) # dim = (batch_size, input_length, hidden_dim)
            log_probs  = F.log_softmax(self.out(hiddens), dim = 2)                    # dim = (batch_size, input_length, lang_size)
            return log_probs

        def computeAccuracy(log_probs, targets) :
            total   = np.sum(targets.data.cpu().numpy() != self.ignore_index)
            success = sum([self.ignore_index != targets[i, j].item() == log_probs[i, :, j].data.topk(1)[1].item() \
                           for i in range(targets.size(0)) \
                           for j in range(targets.size(1)) ])
            return  success * 100 / total

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0

        def trainLoop(batch, optimizer, unmasked_ratio, compute_accuracy = True):
            """Performs a training loop, with forward pass, backward pass and weight update."""
            optimizer.zero_grad()
            self.zero_grad()
            log_probs  = computeLogProbs(batch[0]).transpose(1, 2) # dim = (batch_size, lang_size, input_length)
            targets_xs = batch[1][0].to(self.device)               # dim = (batch_size, input_length)
            targets_xl = batch[1][1].to(self.device)               # dim = (batch_size, input_length)
            loss       = (1-unmasked_ratio)*self.criterion(log_probs, targets_xs) \
                       + unmasked_ratio    *self.criterion(log_probs, targets_xl)
            loss.backward()
            optimizer.step() 
            if compute_accuracy == 'xs': 
                accuracy = computeAccuracy(log_probs, targets_xs)
                error = float(loss.item() / np.sum(targets_xs.data.cpu().numpy() != self.ignore_index))
            elif compute_accuracy == 'xl': 
                accuracy = computeAccuracy(log_probs, targets_xl)
                error = float(loss.item() / np.sum(targets_xl.data.cpu().numpy() != self.ignore_index))
            else : 
                accuracy = 0
                error = float(loss.item() / np.sum(targets_xs.data.cpu().numpy() != self.ignore_index))
            return error, accuracy
        
        # --- main ---
        self.train()
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_acc  = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                batch = random.choice(batches)
                loss, acc = trainLoop(batch, optimizer, unmasked_ratio, compute_accuracy)
                tot_loss += loss
                tot_acc += acc      
                if iter % print_every == 0 : 
                    tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(batches) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(batches)
                for batch in batches :
                    loss, acc = trainLoop(batch, optimizer,unmasked_ratio, compute_accuracy)
                    tot_loss += loss
                    tot_acc += acc 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        return

#### Training

In [38]:
denoiser = SentenceDenoiser(device = torch.device('cpu'), # device
                            tokenizer = lambda s : s.split(' '),
                            word2vec = Word2VecConnector(word2vec),
                            hidden_dim = 75, 
                            n_layers = 3, 
                            dropout = 0.1,
                            optimizer = optim.AdamW)

denoiser.nbParametres()

627164

In [39]:
batches = []
for seed in [42, 854, 7956, 657, 124] :
    batches += denoiser.generatePackedSentences(sentences, 
                                                batch_size = 16,
                                                mask_ratio = 0.15,
                                                max_sentence_length = 50,
                                                tol = 10,
                                                seed = seed)
    print(len(batches))
len(batches)

1830
3660
5490
7320
9150


9150

In [40]:
denoiser.fit(batches, epochs = 1, lr = 0.001, unmasked_ratio = 0.15, compute_accuracy = 'xs', print_every = 100)

epoch 1
0m 14s (- 22m 18s) (100 1%) loss : 10.937  accuracy : 6.3 %
0m 27s (- 20m 35s) (200 2%) loss : 9.902  accuracy : 6.3 %
0m 42s (- 20m 39s) (300 3%) loss : 9.919  accuracy : 7.3 %
0m 55s (- 20m 22s) (400 4%) loss : 9.278  accuracy : 10.1 %
1m 11s (- 20m 31s) (500 5%) loss : 8.866  accuracy : 11.5 %
1m 24s (- 20m 2s) (600 6%) loss : 8.299  accuracy : 13.9 %
1m 38s (- 19m 50s) (700 7%) loss : 7.937  accuracy : 15.7 %
1m 52s (- 19m 36s) (800 8%) loss : 7.376  accuracy : 18.4 %
2m 8s (- 19m 33s) (900 9%) loss : 7.531  accuracy : 17.2 %
2m 21s (- 19m 14s) (1000 10%) loss : 6.964  accuracy : 19.8 %
2m 37s (- 19m 9s) (1100 12%) loss : 6.866  accuracy : 19.4 %
2m 50s (- 18m 50s) (1200 13%) loss : 6.621  accuracy : 20.5 %
3m 4s (- 18m 35s) (1300 14%) loss : 6.507  accuracy : 19.5 %
3m 19s (- 18m 22s) (1400 15%) loss : 6.226  accuracy : 20.9 %
3m 32s (- 18m 4s) (1500 16%) loss : 6.063  accuracy : 22.4 %
3m 46s (- 17m 50s) (1600 17%) loss : 5.990  accuracy : 23.8 %
4m 1s (- 17m 36s) (1700 1

In [41]:
batches2 = []
for seed in [777, 5821, 12, 693, 45824] :
    batches2 += denoiser.generatePackedSentences(sentences, 
                                                batch_size = 16,
                                                mask_ratio = 0.15,
                                                max_sentence_length = 50,
                                                tol = 10,
                                                seed = seed)
    print(len(batches2))
len(batches2)

1830
3660
5490
7320
9150


9150

In [43]:
denoiser.fit(batches2, epochs = 1, lr = 0.0005, unmasked_ratio = 0.15, compute_accuracy = 'xs', print_every = 100)

epoch 1
0m 14s (- 21m 20s) (100 1%) loss : 3.676  accuracy : 38.4 %
0m 26s (- 19m 44s) (200 2%) loss : 3.784  accuracy : 37.8 %
0m 41s (- 20m 21s) (300 3%) loss : 3.828  accuracy : 37.3 %
0m 55s (- 20m 12s) (400 4%) loss : 3.848  accuracy : 38.0 %
1m 7s (- 19m 32s) (500 5%) loss : 3.914  accuracy : 36.7 %
1m 21s (- 19m 24s) (600 6%) loss : 3.530  accuracy : 40.8 %
1m 36s (- 19m 20s) (700 7%) loss : 3.712  accuracy : 39.3 %
1m 50s (- 19m 16s) (800 8%) loss : 3.839  accuracy : 37.2 %
2m 6s (- 19m 19s) (900 9%) loss : 3.756  accuracy : 38.1 %
2m 20s (- 19m 1s) (1000 10%) loss : 3.843  accuracy : 37.3 %
2m 33s (- 18m 45s) (1100 12%) loss : 3.650  accuracy : 39.2 %
2m 48s (- 18m 35s) (1200 13%) loss : 3.868  accuracy : 38.0 %
3m 0s (- 18m 8s) (1300 14%) loss : 3.574  accuracy : 40.6 %
3m 13s (- 17m 51s) (1400 15%) loss : 3.746  accuracy : 37.5 %
3m 26s (- 17m 34s) (1500 16%) loss : 3.936  accuracy : 35.4 %
3m 41s (- 17m 27s) (1600 17%) loss : 3.684  accuracy : 38.2 %
3m 56s (- 17m 15s) (170

In [51]:
batches3 = []
for seed in [124, 5864, 13, 475] :
    batches3 += denoiser.generatePackedSentences(sentences, 
                                                batch_size = 16,
                                                mask_ratio = 0.15,
                                                max_sentence_length = 50,
                                                tol = 10,
                                                seed = seed)
    print(len(batches3))
len(batches3)

1830
3660
5490
7320


7320

In [52]:
denoiser.fit(batches3, epochs = 1, lr = 0.0001, unmasked_ratio = 0.15, compute_accuracy = 'xs', print_every = 100)

epoch 1
0m 15s (- 18m 33s) (100 1%) loss : 3.352  accuracy : 41.6 %
0m 29s (- 17m 18s) (200 2%) loss : 3.182  accuracy : 43.5 %
0m 44s (- 17m 28s) (300 4%) loss : 3.262  accuracy : 41.3 %
0m 58s (- 17m 0s) (400 5%) loss : 3.244  accuracy : 43.1 %
1m 14s (- 16m 54s) (500 6%) loss : 3.125  accuracy : 44.5 %
1m 28s (- 16m 31s) (600 8%) loss : 3.163  accuracy : 44.0 %
1m 42s (- 16m 7s) (700 9%) loss : 3.190  accuracy : 43.4 %
1m 56s (- 15m 52s) (800 10%) loss : 3.329  accuracy : 42.3 %
2m 10s (- 15m 28s) (900 12%) loss : 3.311  accuracy : 41.5 %
2m 25s (- 15m 22s) (1000 13%) loss : 3.192  accuracy : 43.9 %
2m 39s (- 15m 0s) (1100 15%) loss : 3.121  accuracy : 44.5 %
2m 54s (- 14m 48s) (1200 16%) loss : 3.211  accuracy : 42.8 %
3m 8s (- 14m 33s) (1300 17%) loss : 3.232  accuracy : 44.0 %
3m 23s (- 14m 22s) (1400 19%) loss : 3.281  accuracy : 41.6 %
3m 39s (- 14m 9s) (1500 20%) loss : 3.291  accuracy : 42.1 %
3m 53s (- 13m 53s) (1600 21%) loss : 3.226  accuracy : 42.0 %
4m 7s (- 13m 37s) (17

In [78]:
# save
#torch.save(denoiser.state_dict(), path_to_NLP + '\\saves\\models\\DL4NLP_I4a_sentence_denoiser.pth')

# load
#denoiser.load_state_dict(torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I4a_sentence_denoiser.pth'))

#### Evaluation

In [77]:
denoiser.eval()
sentence = ' '.join(sentences[14]) #'what are you thinking of this'
print(sentence)
print('\n')
denoiser(sentence, color = '\033[93m')

reconstitution diluent specific of the product to be analysed


reconstitution diluent volume of the product to be analyzed


In [79]:
denoiser.eval()
sentence = ' '.join(sentences[21]) #'what are you thinking of this'
print(sentence)
print('\n')
denoiser(sentence, color = '\033[93m')

through the vial stopper add the appropriate volume of reconstitution solvent using a syringe fitted with a needle ( diameter smaller than or equal to 0.6 mm )


through the vial stopper add the appropriate volume of reconstitution solvent using a syringe fitted with a needle ( is greater than or equal to mm mm )


#### Word correction

In [29]:
def correctCorpus(word2vec, corpus, threshold = 0.9) :
    new_sentences = []
    corrections = []
    for s in corpus :
        new_s = []
        for word in s :
            try : #fasttext raises an error if no character ngram seen during training appears in the word
                if (word not in word2vec.wv.vocab and word2vec.most_similar(word)[0][1] >= threshold) :
                    new_word = word2vec.most_similar(word)[0][0]
                    new_s.append(new_word)
                    corrections.append([word, new_word])
                else :
                    new_s.append(word)
            except : 
                new_s.append(word)
        new_sentences.append(new_s)
    return new_sentences, corrections


def contextualCorrectCorpus(denoiser, corpus, threshold = 0.9, print_every = 100) :
    word2vec = denoiser.word2vec.word2vec
    new_sentences = []
    corrections = []
    corpus = [s for s in corpus if len(s) > 0]
    for ws in corpus : 
        probs = denoiser.predict_proba(ws).squeeze(0) # size = (input_length, lang_size)
        new_s = []
        for i, word in enumerate(ws) :
            try : #fasttext raises an error if no character ngram seen during training appears in the word
                candidates = [wp[0] for wp in word2vec.most_similar(word) if wp[1] >= threshold]
                if (word not in word2vec.wv.vocab and candidates != []) :
                    indices  = [denoiser.word2vec.lang.getIndex(w) for w in candidates]
                    probsi   = [probs[i, j].item() for j in indices]
                    wps      = [[w, p] for w, p in zip(candidates, probsi)]
                    wps.sort(key = lambda wp : wp[1], reverse = True)
                    new_word = wps[0][0]
                    new_s.append(new_word)
                    corrections.append([word, new_word])
                else :
                    new_s.append(word)
            except : 
                new_s.append(word)
        new_sentences.append(new_s)
        if len(new_sentences)+1 % print_every == 0 : print(len(new_sentences)+1)
    return new_sentences, corrections

In [75]:
corrected_corpus, corrections = correctCorpus(word2vec, corpus, threshold = 0.85)

In [73]:
contextual_corrected_corpus, contextual_corrections = contextualCorrectCorpus(denoiser, corpus[:200], threshold = 0.85)

In [76]:
for i in range(2000) : 
    word  = corrections[i][0]
    pred1 = corrections[i][1]
    pred2 = contextual_corrections[i][1]
    if pred2 != pred1 : pred2 = '\033[93m' + pred2 + '\033[0m'
    print(word, '->', pred1, ' | ', pred2)

riboflavin -> ribosyl  |  ribosyl
22.5cm -> 22.9  |  22.9
return -> returned  |  returned
atmospheric -> atmosphere  |  atmosphere
tracer -> trace  |  trace
2.1.6 -> 2.1.1.1  |  2.1.1.1
duplicated -> duplicate  |  duplicate
interpreting -> interpretation  |  interpretation
pseudoalcaligenes -> pseudomonas  |  sporogenes
coagulogen -> coagulase  |  coagulase
coagulin -> coagulase  |  coagulase
0.0625 -> 0.06  |  0.06
inhibitor -> inhibitory  |  inhibitory
fertilized -> utilized  |  utilized
fulfil -> fulfilled  |  fulfilled
872 -> 72  |  72
872 -> 72  |  72
pharmacovigilance -> pharmacopoeias  |  pharmacopoeia
104.1 -> 104.6  |  104.6
riboflavin -> ribosyl  |  ribosyl
22.5cm -> 22.9  |  22.9
return -> returned  |  returned
atmospheric -> atmosphere  |  atmosphere
tracer -> trace  |  trace
2.1.6 -> 2.1.1.1  |  2.1.1.1
duplicated -> duplicate  |  duplicate
interpreting -> interpretation  |  interpretation
lureus -> aureus  |  aureus
pseudoalcaligenes -> pseudomonas  |  sporogenes
coagulog

pigsa -> pigs  |  pigs
daysa -> days  |  days
daysa -> days  |  days
1.3.1 -> 2.3.1  |  2.3.1
0.003160 -> 0.003  |  0.006
0.13721x -> 0.1315  |  0.1315
0.000643 -> 0.00062  |  0.00062
0.13664x -> 0.1315  |  0.135
0.99999 -> 0.999  |  0.9993
differentiation -> differentiate  |  differentiated
agglutination -> agglutinate  |  hemagglutination
interpreted -> interpretation  |  interpretation
credibility -> compatibility  |  compatibility
throughput -> through  |  through
electronics -> electronically  |  electronically
8.00 -> 8.0  |  8.0
adequately -> adequate  |  adequate
comprising -> comprised  |  comprised
8.00 -> 8.0  |  8.0
deliberately -> deliberate  |  deliberate
22.3oc -> 22.9  |  22.9
3.2.p.3.2 -> 3.2.p.3.4  |  3.2.p.3.3
dtapa -> dtap  |  dtap
improvement -> improve  |  improve
2.7.2 -> 2.7.20  |  2.7.20
2.7.3 -> 2.7.6  |  2.7.8
2.7.4 -> 2.7.6  |  2.7.16
2.7.1.1 -> 2.7.1  |  2.7.1
2.7.1.2 -> 2.7.1  |  2.7.1
avoidance -> avoid  |  avoid
rejecting -> reject  |  rejection
retransf

58.1 -> 8.1  |  8.1
ameboecyte -> amebocyte  |  limulus
allowable -> allow  |  allow
describe -> described  |  described
hexaxim -> hexaxim  |  hexaxim
concentration -> concentrat  |  concentrat
concentration -> concentrat  |  concentrat
0.26 -> 0.2698  |  0.2698
0.105 -> 0.100  |  0.100
processor -> process  |  process
classify -> classified  |  classified
classification -> classified  |  classified
aemophilus -> haemophilus  |  haemophilus
hexaxim -> hexaxim  |  hexaxim
2.2.1.3 -> 2.2.1  |  2.2.1
71.2 -> 1.2  |  1.2
2.6.2 -> 2.6.9  |  2.6.1
71.2 -> 1.2  |  1.2
2.4.1.2 -> 2.4.18  |  2.4.18
2.4.1.2 -> 2.4.18  |  2.4.18
1.7861 -> 1.7  |  1.7
bioanalytical -> analytical  |  analytical
ewp -> dtwp  |  dtwp
haemaglutinin -> haemagglutinin  |  haemagglutinin
haemaglutinin -> haemagglutinin  |  haemagglutinin
constraints -> constrained  |  constrained
constraints -> constrained  |  constrained
experimentally -> experimental  |  experiment
ctpa -> ctp  |  ctp
12000 -> 2000  |  2000
haemag

IndexError: list index out of range